In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loading

In [ ]:
!unzip drive/MyDrive/"Dataset #2".zip

Archive:  drive/MyDrive/Dataset #2.zip
   creating: Dataset #2/
   creating: Dataset #2/test/
   creating: Dataset #2/test/Inertial Signals/
  inflating: Dataset #2/test/Inertial Signals/test_acc_x.txt  
  inflating: Dataset #2/test/Inertial Signals/test_acc_y.txt  
  inflating: Dataset #2/test/Inertial Signals/test_acc_z.txt  
  inflating: Dataset #2/test/Inertial Signals/test_gyr_x.txt  
  inflating: Dataset #2/test/Inertial Signals/test_gyr_y.txt  
  inflating: Dataset #2/test/Inertial Signals/test_gyr_z.txt  
  inflating: Dataset #2/test/y_test.txt  
   creating: Dataset #2/train/
   creating: Dataset #2/train/Inertial Signals/
  inflating: Dataset #2/train/Inertial Signals/train_acc_x.txt  
  inflating: Dataset #2/train/Inertial Signals/train_acc_y.txt  
  inflating: Dataset #2/train/Inertial Signals/train_acc_z.txt  
  inflating: Dataset #2/train/Inertial Signals/train_gyr_x.txt  
  inflating: Dataset #2/train/Inertial Signals/train_gyr_y.txt  
  inflating: Dataset #2/train/Inert

In [ ]:
!unzip drive/MyDrive/Amna_Train.zip

Archive:  drive/MyDrive/Amna_Train.zip
   creating: Amna_Train/
  inflating: Amna_Train/acc_train_x.txt  
  inflating: Amna_Train/acc_train_y.txt  
  inflating: Amna_Train/acc_train_z.txt  
  inflating: Amna_Train/gyro_train_x.txt  
  inflating: Amna_Train/gyro_train_y.txt  
  inflating: Amna_Train/gyro_train_z.txt  


In [ ]:
!unzip drive/MyDrive/Amna_Test.zip


Archive:  drive/MyDrive/Amna_Test.zip
   creating: Amna_Test/
  inflating: Amna_Test/acc_test_x.txt  
  inflating: Amna_Test/acc_test_y.txt  
  inflating: Amna_Test/acc_test_z.txt  
  inflating: Amna_Test/gyro_test_x.txt  
  inflating: Amna_Test/gyro_test_y.txt  
  inflating: Amna_Test/gyro_test_z.txt  


# Data PreProcessing

In [ ]:
def load_X(path):
    X_signals = []
    files = os.listdir(path)
    files.sort(key=str.lower)
    #['train_acc_x.txt', 'train_acc_y.txt', 'train_acc_z.txt', 'train_gyr_x.txt', 'train_gyr_y.txt', 'train_gyr_z.txt']
    for my_file in files:
        fileName = os.path.join(path,my_file)
        file = open(fileName, 'r')
        X_signals.append(
            [np.array(cell, dtype=np.float32) for cell in [
                row.strip().split(' ') for row in file
            ]]
        )
        file.close()
        #X_signals = 6*totalStepNum*128
    X_signals = np.transpose(np.array(X_signals), (1, 0, 2))#(totalStepNum*6*128)
    return X_signals.reshape(-1,6,128,1)#(totalStepNum*6*128*1)

In [ ]:
def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    y_ = y_ - 1
    #one_hot
    y_ = y_.reshape(len(y_))
    n_values = int(np.max(y_)) + 1
    return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

# Model Construction

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)

    return tf.Variable(initial)

In [ ]:
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [ ]:
batch_size = 512
X_ = tf.placeholder(tf.float32, [None, 6, 128, 1],name='cnn_X')
label_ = tf.placeholder(tf.float32, [None, 21],name='cnn_Y')


In [ ]:
import tensorflow.keras.layers as layers
from tensorflow.keras.models import save_model
#input shape [batch, in_height, in_width, in_channels]
#kernel shape [filter_height, filter_width, in_channels, out_channels]
'''
	1*9
	stride = 2
	padding
	6*128->6*64*32
'''
W_conv1 = weight_variable([1, 9, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(
    tf.nn.conv2d(X_, W_conv1, strides=[1, 1, 2, 1], padding='SAME') + b_conv1)
'''
	pooling
	6*64*32->6*32*32
'''
h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 1, 2, 1], strides=[1, 1, 2, 1], padding='VALID')
'''
	1*3
	stride = 1
	6*32*32->6*32*64
'''
W_conv2 = weight_variable([1, 3, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(
    tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
'''
	1*3
	stride = 1
	padding
	6*32*64->6*32*128
'''

W_conv3 = weight_variable([1, 3, 64, 128])
b_conv3 = bias_variable([128])
h_conv3 = tf.nn.relu(
    tf.nn.conv2d(h_conv2, W_conv3, strides=[1, 1, 1, 1], padding='SAME') + b_conv3)

'''
'''
'''
	pooling
	6*32*128->6*16*128
'''
h_pool2 = tf.nn.max_pool(h_conv3, ksize=[1, 1, 2, 1], strides=[1, 1, 2, 1],padding='VALID')
'''
	6*1
	6*32*128->1*16*128
'''
W_conv4 = weight_variable([6, 1, 128, 128])
b_conv4 = bias_variable([128])
h_conv4 = tf.nn.relu(
    tf.nn.conv2d(h_pool2, W_conv4, strides=[1, 1, 1, 1], padding='VALID') + b_conv4)
'''
	input flat 16*128=2048
	output 20
'''
h_flat = layers.Flatten()(h_conv4)
cnn_output = tf.multiply(h_conv4,1,name='cnn_output')

In [ ]:
W_fc = weight_variable([2048, 21])
b_fc = bias_variable([21])
h_fc = tf.nn.softmax(tf.matmul(h_flat, W_fc) + b_fc)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(label_ * tf.log(h_fc+1e-10), reduction_indices=[1]),name='cnn_loss')
train_step = tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(h_fc, 1), tf.argmax(label_, 1),name='cnn_pre_Y')
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32),name='cnn_accuracy')


In [ ]:
X_train = load_X('Dataset #2/train/Inertial Signals/')
X_test = load_X('Dataset #2/test/Inertial Signals/')


train_label = load_y('Dataset #2/train/y_train.txt')
test_label = load_y('Dataset #2/test/y_test.txt')

In [ ]:
X_train_enroll = load_X('/content/Amna_Train/')
X_test_enroll = load_X('/content/Amna_Test/')

In [ ]:
merged_train = np.concatenate((X_train, X_train_enroll), axis=0)

In [ ]:
merged_test = np.concatenate((X_test , X_test_enroll), axis=0)

In [ ]:
merged_train.shape

(44791, 6, 128, 1)

In [ ]:
merged_test.shape


(4985, 6, 128, 1)

In [ ]:
X_test.shape

(4936, 6, 128, 1)

In [ ]:
test_label.shape

(4985, 21)

In [ ]:
X_train.shape

(44339, 6, 128, 1)

In [ ]:
train_label.shape

(44791, 21)

# Model Training

In [ ]:
saver = tf.train.Saver()


In [ ]:
sess.close()

In [ ]:
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [ ]:
history = dict(train_loss=[],
                     train_acc=[],
                     test_loss=[],
                     test_acc=[])

In [ ]:
#if os.path.exists('/content/drive/MyDrive/dataset2_model'):
 #   saver.restore(sess,tf.train.latest_checkpoint('/content/drive/MyDrive/dataset2_model/'))

best_accuracy = 0

for i in range(200):
    l = len(train_label)
    batch_idxs = int(l / batch_size)
    index = list(range(l))
    random.shuffle(index)
    for idx in range(batch_idxs):
        image_idx = merged_train[index[idx * batch_size:(idx + 1) * batch_size]]
        label_idx = train_label[index[idx * batch_size:(idx + 1) * batch_size]]
        #print(start,end)
        acc, loss, _ = sess.run([accuracy, cross_entropy, train_step], feed_dict={
            X_: image_idx,
            label_: label_idx
        })
        if idx % 100 == 0:
            print(str(i) + 'the cross_entropy:', str(loss), 'train_accuracy:', str(acc))

        # Test completely at every epoch: calculate accuracy
    accuracy_out, loss_out = sess.run(
        [accuracy, cross_entropy],
        feed_dict={
            X_: merged_test,
            label_: test_label
        }
    )
    history['train_loss'].append(loss)
    history['train_acc'].append(acc)
    history['test_loss'].append(loss_out)
    history['test_acc'].append(accuracy_out)
    if accuracy_out > best_accuracy:
        saver.save(sess,'/content/drive/MyDrive/dataset2_model')
        #tf.train.write_graph(sess.graph_def, '.', '/content/drive/MyDrive/CNN_Graph_dataset2_model.pbtxt')
        best_accuracy = accuracy_out

    print(str(i)+'--------------the cross_entropy:', str(loss_out), '-----------------------test_accuracy:', str(accuracy_out))

print("best accuracy:"+str(best_accuracy))


0the cross_entropy: 7.39621 train_accuracy: 0.046875
0--------------the cross_entropy: 0.5742482 -----------------------test_accuracy: 0.90290874
1the cross_entropy: 0.11791617 train_accuracy: 0.97265625
1--------------the cross_entropy: 0.38639742 -----------------------test_accuracy: 0.9412237
2the cross_entropy: 0.084640995 train_accuracy: 0.9765625
2--------------the cross_entropy: 0.39585426 -----------------------test_accuracy: 0.95205617
3the cross_entropy: 0.038147457 train_accuracy: 0.9902344
3--------------the cross_entropy: 0.34147975 -----------------------test_accuracy: 0.952658
4the cross_entropy: 0.0769468 train_accuracy: 0.9824219
4--------------the cross_entropy: 0.3549653 -----------------------test_accuracy: 0.96128386
5the cross_entropy: 0.015040932 train_accuracy: 0.9980469
5--------------the cross_entropy: 0.37313166 -----------------------test_accuracy: 0.9542628
6the cross_entropy: 0.020622713 train_accuracy: 0.9941406
6--------------the cross_entropy: 0.3535184

In [ ]:
tf.InteractiveSession().close()

In [ ]:
sess.close()

# Model Evaluation

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, '/content/drive/MyDrive/dataset2_model')

   # sess = tf.InteractiveSession(config=config)
    #sess.run(tf.global_variables_initializer())
    # Restore the saved model parameters
    #saver = tf.train.Saver()
    #tf.train.Saver().restore(sess, '/content/drive/MyDrive/dataset2_model')

    # Now you can perform inference on a specific test sample
    # Select the index of the test sample you want to use
    sample_index = 4740  # Specify the index of the test sample you want to use (e.g., 0, 1, 2, ...)

    # Retrieve the corresponding test sample and label
    test_sample = merged_test[sample_index]
    sample_label = test_label[sample_index]

    # Reshape the test sample to match the input shape of the model
    test_sample = test_sample.reshape((1, 6, 128, 1))

    # Feed the test sample into the model
    prediction = tf.argmax(h_fc, 1)
    test_result = sess.run(prediction, feed_dict={X_: test_sample})

    sample_decode=np.array(sample_label)
    decoded_value = np.argmax(sample_decode)

    # Use the test result for further analysis or evaluation
    print("Test Result:", test_result)
    print("True Label:", decoded_value)

Test Result: [4]
True Label: 4


# Conversion to TensorFlow Lite Model

In [ ]:
from tensorflow.python.tools import freeze_graph


input_graph_path = '/content/drive/MyDrive/CNN_Graph.pbtxt'
checkpoint_path = '/content/drive/MyDrive/cnn_ckpt_updated'
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = '/content/drive/MyDrive/frozen_cnn.pb'

freeze_graph.freeze_graph(input_graph_path, input_saver="",
                          input_binary=False, input_checkpoint=checkpoint_path,
                          output_node_names="cnn_output", restore_op_name="save/restore_all",
                          filename_tensor_name="save/Const:0",
                          output_graph=output_frozen_graph_name, clear_devices=True, initializer_nodes="")

In [ ]:
input_pb_file = '/content/drive/MyDrive/frozen_cnn.pb'


# Path to the output .tflite file
output_tflite_file = '/content/drive/MyDrive/model.tflite'

# Load the frozen graph
graph_def = tf.compat.v1.GraphDef()
with tf.io.gfile.GFile(input_pb_file, 'rb') as f:
    graph_def.ParseFromString(f.read())

# Convert the frozen graph to TensorFlow Lite format
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph('/content/drive/MyDrive/frozen_cnn.pb', input_arrays=["cnn_X"], output_arrays=["cnn_output/y"])
tflite_model = converter.convert()

# Save the converted .tflite model to disk
with open(output_tflite_file, 'wb') as f:
    f.write(tflite_model)

In [ ]:
input_pb_file = 'frozen_cnn.pb'

# Load the frozen graph
graph_def = tf.compat.v1.GraphDef()
with tf.io.gfile.GFile(input_pb_file, 'rb') as f:
    graph_def.ParseFromString(f.read())

# Print the input and output tensor names
with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name='')
    for op in graph.get_operations():
        print('Operation:', op.name)
        for output in op.outputs:
            print('Output:', output.name)

Operation: cnn_X
Output: cnn_X:0
Operation: Variable
Output: Variable:0
Operation: Variable/read
Output: Variable/read:0
Operation: Variable_1
Output: Variable_1:0
Operation: Variable_1/read
Output: Variable_1/read:0
Operation: Conv2D
Output: Conv2D:0
Operation: add
Output: add:0
Operation: Relu
Output: Relu:0
Operation: MaxPool
Output: MaxPool:0
Operation: Variable_2
Output: Variable_2:0
Operation: Variable_2/read
Output: Variable_2/read:0
Operation: Variable_3
Output: Variable_3:0
Operation: Variable_3/read
Output: Variable_3/read:0
Operation: Conv2D_1
Output: Conv2D_1:0
Operation: add_1
Output: add_1:0
Operation: Relu_1
Output: Relu_1:0
Operation: Variable_4
Output: Variable_4:0
Operation: Variable_4/read
Output: Variable_4/read:0
Operation: Variable_5
Output: Variable_5:0
Operation: Variable_5/read
Output: Variable_5/read:0
Operation: Conv2D_2
Output: Conv2D_2:0
Operation: add_2
Output: add_2:0
Operation: Relu_2
Output: Relu_2:0
Operation: MaxPool_1
Output: MaxPool_1:0
Operation: V

In [ ]:
g = tf.GraphDef()
g.ParseFromString(open('/content/drive/MyDrive/frozen_cnn.pb', 'rb').read())
print(g)

node {
  name: "cnn_X"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: -1
        }
        dim {
          size: 6
        }
        dim {
          size: 128
        }
        dim {
          size: 1
        }
      }
    }
  }
}
node {
  name: "Variable"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 1
          }
          dim {
            size: 9
          }
          dim {
            size: 1
          }
          dim {
            size: 32
          }
        }
        tensor_content: "\322\000_\274\276]\253<m\002\016\276\371\253@=\017\':=\255\036\272\275+H\253\275\362\014\026\276 \334\201\271\213\200\336\273id\010\276\033\270/=\362\276C>\272$\256=\']\276\275\036p\031>2\014l\275H\271\347\27